#### Seq2Seq


![image](../data/image/seq2seq.jpg)

![image](../data/image/EncoderDecoder.jpg)

模型中有两个主要组件
- 编码器（Encoder）：负责将输入序列转换为固定大小的向量表示，编码器采用RNN、LSTM或GRU等模型，编码器会逐个处理输入序列中的元素，在每个时间步更新其隐藏状态，最后，编码器会生成一个上下文向量，它包含了整个输入序列的信息。
- 解码器（Decoder）：负责将编码器生成的上下文向量转换为输出序列。解码器采用RNN、LSTM或GRU等模型，使用来自编码器的上下文向量作为其初始隐藏状态，并逐个生成输出序列中的元素。在每个时步，解码器根据当前隐藏状态、生成的上一个输出元素及其他可能的信息，来生成下一个输出的元素



#### 构建演示语料库和词汇表

数据说明：
eg. '小明 喜欢 小冰', '<SOS> XiaoMing likes XiaoBing','XiaoMing likes XiaoBing <EOS>'
    
- 第一句，中文句子，作为输入序列提供给编码器
- 第二句，(<SOS>+目标语言)：英文句子，作为解码器的输入序列。句子以特殊符号<SOS>开头，表示句子的开始。<SOS>有助于解码器学会在何时开始生成目标句子。
- 第三句，(目标语言+<EOS>)：也是英文句子，作为解码器的目标输出序列，句子以特殊符号<EOS>结尾，表示句子结束。<EOS>有助于解码器学会在何时结束目标句子的生成。

In [1]:

sentences = [
    ['小明 喜欢 小冰', '<SOS> XiaoMing likes XiaoBing','XiaoMing likes XiaoBing <EOS>'],
    ['我 爱 学习 人工智能', '<SOS> I Love studying AI', 'I Love studying AI <EOS>'],
    ['深度学习 改变 世界', '<SOS> DL changed the world', 'DL changed the world <EOS>'],
    ['自然 语言 处理 很 强大', '<SOS> NLP is so powerful', 'NLP is so powerful <EOS>'],
    ['神经网络 非常 复杂', '<SOS> Neural-Nets are complex', 'Neural-Nets are complex <EOS>']
]
# 初始化中英文词汇表
word_list_cn, word_list_en = [],[]

for sent in sentences:
    word_list_cn.extend(sent[0].split())
    word_list_en.extend(sent[1].split())
    word_list_en.extend(sent[2].split())

word_list_cn = list(set(word_list_cn))
word_list_en = list(set(word_list_en))

word2idx_cn = {word:idx for idx, word in enumerate(word_list_cn)}
word2idx_en = {word:idx for idx, word in enumerate(word_list_en)}

idx2word_cn = {idx:word for idx, word in enumerate(word_list_cn)}
idx2word_en = {idx:word for idx, word in enumerate(word_list_en)}

voc_size_cn = len(word_list_cn)
voc_size_en = len(word_list_en)

#### 生成Seq2Seq训练数据

为什么要有解码器输入张量？  
在训练阶段，向解码器提供这个信息，模型就能够以正确单词为基础来生成下一个单词，以提高训练速度

In [2]:
import random
import numpy as np
import torch

In [3]:
def make_data(sentences):
    #从数据中随机抽取一对数据
    random_sentence = random.choice(sentences)
    #对编码器输入数据序列进行编码
    encoder_input = [word2idx_cn[sen] for sen in random_sentence[0].split()]
    #对解码器输入数据序列进行编码
    decoder_input = [word2idx_en[sen] for sen in random_sentence[1].split()]
    #对编码器输入数据序列进行编码
    target = [word2idx_en[sen] for sen in random_sentence[2].split()]
    
    encoder_input = torch.LongTensor(encoder_input).unsqueeze(0)
    decoder_input = torch.LongTensor(decoder_input).unsqueeze(0)
    target = torch.LongTensor(target).unsqueeze(0)
    return encoder_input, decoder_input, target

In [5]:
encoder_input, decoder_input, target = make_data(sentences)
# print("原始句子:", random_sentence)
print("编码器输入张量: ", encoder_input)
print("解码器输入张量: ", decoder_input)
print("目标张量: ", target)

编码器输入张量:  tensor([[ 0, 14, 17]])
解码器输入张量:  tensor([[18, 12,  8, 15,  4]])
目标张量:  tensor([[12,  8, 15,  4, 16]])


#### 构建编码器和解码器

In [6]:
import torch.nn as nn

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        #词嵌入 word embedding
        self.embedd = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, inputs, hidden):
        embedded = self.embedd(inputs)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

In [38]:
encoder = Encoder(voc_size_cn, 128)
input = torch.LongTensor([[1,2,3,4]])
hidden = torch.zeros(1,input.size(0),128)
output,hiddens= encoder(input, hidden)
print(output.shape, hiddens.shape)

torch.Size([1, 4, 128])
torch.Size([1, 4, 128]) torch.Size([1, 1, 128])


In [8]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.embedd = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, inputs, hidden):
        embedded = self.embedd(inputs)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output)
        return output, hidden

In [41]:
decoder = Decoder(128, voc_size_en)
input = torch.LongTensor([[1,2,3,4]])
# hidden = torch.zeros(1,input.size(0),128)
output,hiddens= decoder(input, hiddens)
print(output.shape, hiddens.shape)

torch.Size([1, 4, 20]) torch.Size([1, 1, 128])


In [9]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, enc_input, hidden, dec_input):
        encoder_output, encoder_hidden = self.encoder(enc_input, hidden)
        decoder_hidden = encoder_hidden
        decoder_output, _ = self.decoder(dec_input, decoder_hidden)
        return decoder_output
        

#### 训练Seq2Seq模型

In [10]:
def train_seq2seq(model, criterion, optimizer, epochs):
    '''
    model：模型
    criterion：损失函数
    optimizer：优化器
    epochs：迭代次数
    '''
    for epoch in range(epochs):
        encoder_input, decoder_input, target = make_data(sentences)
        hidden = torch.zeros(1, encoder_input.size(0), n_hidden)
        optimizer.zero_grad()
        output = model(encoder_input, hidden, decoder_input)
        loss = criterion(output.view(-1,voc_size_en), target.view(-1))
        if(epoch + 1) % 40 == 0:
            print(f'Epoch:{epoch+1:04d} cost = {loss:6f}')
        loss.backward()
        optimizer.step()  

In [11]:
epochs = 400
n_hidden = 128
criterion = nn.CrossEntropyLoss()
encoder = Encoder(voc_size_cn, n_hidden)
decoder = Decoder(n_hidden, voc_size_en)
model = Seq2Seq(encoder, decoder)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)
train_seq2seq(model,criterion,optimizer,epochs)

Epoch:0040 cost = 0.839249
Epoch:0080 cost = 0.059344
Epoch:0120 cost = 0.048464
Epoch:0160 cost = 0.035758
Epoch:0200 cost = 0.016393
Epoch:0240 cost = 0.013099
Epoch:0280 cost = 0.015326
Epoch:0320 cost = 0.010872
Epoch:0360 cost = 0.009402
Epoch:0400 cost = 0.007468


#### 测试，使用训练的Seq2Seq模型

In [38]:
def test_seq2seq(model, source_sentence):
    encoder_input = torch.LongTensor([word2idx_cn[n] for n in source_sentence.split()])
    decoder_input = torch.LongTensor([word2idx_en['<SOS>']] + [word2idx_en['<EOS>']] * (len(encoder_input)-1))
    encoder_input = encoder_input.unsqueeze(0)
    decoder_input = decoder_input.unsqueeze(0)
    hidden = torch.zeros(1, encoder_input.size(0), n_hidden)
    predict = model(encoder_input, hidden, decoder_input)
    predict = predict.data.max(dim=2, keepdim=True)[1]
    print(source_sentence, '->',[idx2word_en[n.item()] for n in predict.squeeze()])

In [42]:
predict = test_seq2seq(model, '小明 喜欢 小冰')

小明 喜欢 小冰 -> ['XiaoMing', 'likes', 'XiaoBing']
